In [1]:
import pandas as pd
import re
import spacy

from adept.config import DATA_DIR, TRAINING_DIR, PROCESSED_DATA_DIR
from adept.preprocess import Preprocess
from adept.postprocess import Postproccess
import spacy
from spacy.language import Language
from spacy.tokens import Doc
from adept.utils.helpers import token_get_ent
from adept.components.registry import ComponentsRegistry
from spacy import displacy
import re
from tokenize import Token
from spacy.matcher import Matcher
from spacy.tokens import Span

from adept.utils.helpers import token_get_ent
from adept.config import logger, RAW_DATA_DIR

/Users/ben/Projects/NaturalHistoryMuseum/ADEPT/ADEPT/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nlp = spacy.load(TRAINING_DIR / 'adept')

In [3]:
registry = ComponentsRegistry(nlp)

In [4]:
registry.add_component('anatomical_ner')
registry.add_component('traits_ner')
registry.add_component('traits_custom_ner')
registry.add_component('numeric', after='traits_custom_ner')

In [7]:
preprocess = Preprocess()
# nlp = spacy.load('en_core_web_trf')
nlp.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x1466e92a0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1466f0430>),
 ('sentencizer',
  <adept.components.sentencizer.SentencizerComponent at 0x1466e60b0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1466e9300>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1466f0510>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x146710080>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1466ba040>),
 ('anatomical_ner',
  <adept.components.anatomical.AnatomicalComponent at 0x146739b00>),
 ('traits_ner',
  <adept.components.traits.DiscreteTraitsComponent at 0x145101300>),
 ('traits_custom_ner',
  <adept.components.traits.CustomTraitsComponent at 0x10ea5ae90>),
 ('numeric', <adept.components.numeric.NumericComponent at 0x10eb99480>)]

In [6]:
df = pd.read_csv(PROCESSED_DATA_DIR / 'af4b0c64764555e29e5f5a7847683c2f.descriptions.csv')

In [36]:
# row = df.sample()
# description = row['description'].item()

# for row in df.itertuples():
#     description = row.description
#     text = preprocess(description)  
#     doc = nlp(text)    
#     break

description = 'Trees to 10 m tall, andromonoecious. Branchlets stout, with elliptic or oblong lenticels; winter buds ovoid, outer scales ovate. Leaves deciduous; petiole red, 5-9 cm, glabrous or pubescent; leaf blade abaxially light green, adaxially dark green, 8-12 cm wide, membranous, abaxially yellowish pilose, adaxially glabrous except pubescent on veins, base cordate or deeply cordate, 5-lobed, rarely 7-lobed; lobes ovate or triangular-ovate, margin doubly and sharply incised-serrate, with acuminate teeth, apex caudate or acuminate. Inflorescence terminal, red, compact, racemose-paniculate, villous; peduncles 8-10(-12) cm. Pedicel 5(-8) mm, slender, slightly pubescent. Sepals 5, ovate-lanceolate, ca. 3 mm. Petals 5, linear-oblong, ca. 7 mm, glabrous. Stamens 8, glabrous. Disk extrastaminal, lobed, glabrous. Ovary densely yellow tomentose; style 1-5 mm; stigmas spreading. Fruit yellowish brown; nutlets ovoid-subglobose, rarely flat, veined; wing including nutlet 2.5-2.8 cm × 7-9 mm, wings spreading acutely or erectly. Fl. May, fr. Sep. 2n=26.'

# description = 'Herbs to 40-100 cm tall, annual, much branched. 2 ovaries. 56 stamenoids. Seed volume is about 2 cm³. 2n=23,34'
text = preprocess(description)
doc = nlp(text)

# description = row['description'].item()

    # break
# description

In [37]:


for ent in doc.ents:
    print(ent)
    print(ent.label_)

Trees
TRAIT
10 m
MEASUREMENT
andromonoecious
TRAIT
Branchlets
TRAIT
elliptic
TRAIT
oblong
TRAIT
buds
PART
ovoid
TRAIT
scales
TRAIT
ovate
TRAIT
Leaves
PART
deciduous
TRAIT
petiole
TRAIT
red
COLOUR
5-9 cm
MEASUREMENT
glabrous
TRAIT
pubescent
TRAIT
leaf
PART
blade
PART
green
COLOUR
green
COLOUR
8-12 cm
MEASUREMENT
membranous
TRAIT
yellowish
COLOUR
pilose
TRAIT
glabrous
TRAIT
pubescent
TRAIT
veins
PART
cordate
TRAIT
cordate
TRAIT
lobed
TRAIT
lobed
TRAIT
lobes
PART
ovate
TRAIT
triangular
TRAIT
ovate
TRAIT
serrate
TRAIT
acuminate
TRAIT
caudate
TRAIT
acuminate
TRAIT
Inflorescence
PART
terminal
TRAIT
red
COLOUR
racemose
TRAIT
paniculate
TRAIT
villous
TRAIT
peduncles
PART
8-10(-12) cm
MEASUREMENT
Pedicel
PART
5(-8) mm
MEASUREMENT
pubescent
TRAIT
Sepals
PART
5
CARDINAL
ovate-lanceolate
TRAIT
ca. 3 mm
MEASUREMENT
Petals
PART
5
CARDINAL
linear-oblong
TRAIT
ca. 7 mm
MEASUREMENT
glabrous
TRAIT
Stamens
PART
8
CARDINAL
glabrous
TRAIT
Disk
PART
lobed
TRAIT
glabrous
TRAIT
Ovary
PART
yellow
COLOUR
toment

In [10]:
for ent in doc.ents:
    x = ent

In [11]:
for token in x:
    print(x)
    print(x.ents)
    print(x.ent_id_)

2n=23,34
[2n=23,34]



In [39]:
displacy.render(doc, style="ent")

In [41]:
ents = [ent for ent in doc.ents if ent.label_ == 'CARDINAL']
for cardinal_ent in ents:
    token = cardinal_ent.root
    
    print(cardinal_ent)
    print(cardinal_ent._.numeric_value)
    
    if token.dep_ == 'nummod':
        ent = token_get_ent(token.head, ['PART', 'TRAIT']) 

    
    
    # print(token.head)
    # print(ent.children)
    # print(dir(root))
    # displacy.render(ent.sent, style="dep")
    
    
    
    
    

5
5
5
5
8
8


In [22]:
from spacy.tokens import Span, Doc

from adept.traits import Traits
from adept.fields import Fields
from adept.utils.helpers import token_get_ent


In [83]:
from abc import ABC, abstractmethod
from collections import OrderedDict
from pathlib import Path
from spacy.tokens import Span
import re
import yaml

from adept.config import unit_registry, logger
from adept.utils.helpers import flatten_dict
from decimal import Decimal, InvalidOperation

        
class Field(ABC):
    
    unique = False
    
    def __init__(self, name):
        self.name = name
        self.value = set()

    @property
    @abstractmethod
    def field_type(self):
        pass        
        
    def set_value(self, value):
        self.value.add(value)
    
    def get_value(self):
        return ', '.join(self.value)
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.value})'
    
class NumericField(Field):
    
    num_re = re.compile(r'(\d+(?:\.\d+)?)')
    field_type = 'numeric'
    
    def set_value(self, ent: Span):
        self.value = self._get_ent_value(ent)
        
    def get_value(self):
        return self.value
    
    def _get_ent_value(self, ent: Span):
        if ent._.numeric_range:
            value = ent._.numeric_range
        else:
            value = {'from': ent._.numeric_value, 'to': ent._.numeric_value}    
        return self._to_min_max(value)


    def _to_min_max(self, value_dict):
        unpack = lambda ks: ([v for k in ks if (v := value_dict.get(k))])
        return {
            'min': min(unpack(['lower', 'from']), default=None),
            'max': max(unpack(['to', 'upper']), default=None)
        }  

class MeasurementField(NumericField):
    
    field_type = 'measurement'
    
    def __init__(self, name):
        self.name = name
        self.value = {}
    
    # Length/height measurements are provided first, followed by width. 
    dimension_axes = ['y', 'x']    

    def set_value(self, ent, axis = None):   
        
        if len(self.value) == 2:
            logger.error(f'Field {self.name} already has a value')
            return
        
        if not axis:    
            axis = self.dimension_axes[len(self.value)]

        self._set_value(ent, axis)
        
    def get_value(self, target_unit=None):
        data = {}        
        for axis in self.dimension_axes:
            if axis in self.value: 
                data[axis] = {k: self._format_output(v, target_unit) for k, v in self.value.get(axis).items() if v}
        return data
    
    def _format_output(self, value, target_unit=None):
        if target_unit:
            value = value.to(target_unit)

        # Convert value to string - uses the default formatting set in adept.config unit_registry.default_format  
        return f'{value}'

    def _set_value(self, ent, axis):                  
        unit = ent._.unit        
        # Some measurements are detected, but have no unit. 
        # E.g. Petals white, suborbicular, 6-7 x 5-6.
        # No unit = do not use the measurement                
        if not unit:
            logger.error(f'No unit detected for measurement field {self.name} {ent.text}')
            return
        
        value_dict = self._get_ent_value(ent)
        value_dict = {x: self._to_unit(y, unit) for x, y in value_dict.items() if y}
        self.value[axis] = value_dict        
        
    def _to_unit(self, value, unit):
        if value:
            return value * unit
         

class DimensionField(MeasurementField):
    
    def set_value(self, ent: Span):   
        ents = ent._.get("dimension_parts")        
        for axis, dim_ent in zip(self.dimension_axes, ents):        
            self._set_value(dim_ent, axis)


class VolumeField(MeasurementField):   
    def _set_value(self, ent: Span):          
        self.value = self._get_minmax_value(ent, ent._.measurement_unit)

    def get_value(self, target_unit=None):
        if unit:
            unit = unit_registry(unit)        
        data = {}        
        for mm, value in self.value.items():
            if unit:
                value = self.convert_value(value, unit)  
            data[mm] = self._to_string(value)
        if minmax: data = data[minmax]
        return data    
    
            
class DiscreteField(Field):
    
    field_type = 'discrete'



        
class Fields(object):
    
    _classes = {
        'discrete': DiscreteField,
        'measurement': MeasurementField,
        'dimension': DimensionField,
        'numeric': NumericField,
        'volume': VolumeField,
    }    
    
    re_unit = re.compile('\[([a-z³]+)\]')
    
    def __init__(self):
        self._fields = OrderedDict()
        
    def upsert(self, field_name, field_type, value):
        # If we do not already have the field defined create it         
        if not field_name in self._fields: 
            self._fields[field_name] = self._factory(field_type, field_name)

        self._fields[field_name].set_value(value)
            
    def _factory(self, field_type, field_name):
        return self._classes[field_type](field_name)  
    
    def to_dict(self, field_configs={}):
        data = OrderedDict()
        for field in self._fields.values():
            field_config = field_configs.get(field.name, {})
            if value := field.get_value(**field_config):            
                value_dict = {field.name: value}
                if isinstance(value, dict):
                    # We want to turn measurement dicts etc into single dimension dicts e.g.
                    # {'field_name': {'y': {'min': 40.0, 'max': 100.0}}} => {'field_name.y.min': 40.0, 'field_name.y.max': 100.0}
                    value_dict = flatten_dict(value_dict)
                data.update(value_dict)   
        return data   
    
    def to_template_dict(self, template_path: Path):

        with template_path.open('r') as f:
            field_mappings = yaml.full_load(f)         
        
        field_configs = {field_name.split('.')[0]: {'target_unit': unit} for template_name, field_name in field_mappings.items() if (unit := self.extract_unit(template_name))}
        data_dict = self.to_dict(field_configs)
        
        def _get_value(field_name):
            field_names = field_name if isinstance(field_name, list) else [field_name] 
            for field_name in field_names:
                if value := data_dict.get(field_name):
                    return value
        
        return OrderedDict([(template_name, _get_value(field_name or template_name)) for template_name, field_name in field_mappings.items()])

    def extract_unit(self, string):        
        if match := self.re_unit.search(string):
            return match.group(1)
    
    

class Postproccess():
    
    """
    
    Run through the description Doc, parsing the different entities

    Returns:
        _type_: _description_
    """
    
    traits = Traits()
    
    def __call__(self, doc, taxon_group):
        fields = Fields()
        for sent in doc.sents:
            part = self._sent_get_part(sent)            
            self._process_discrete_fields(sent, fields, part, taxon_group)
            self._process_custom_fields(sent, fields)
            if part:
                self._process_measurement_fields(sent, fields, part)
                self._process_colours_fields(sent, fields, part)
            self._process_numeric_fields(sent, fields)
                
        return fields
          
    @staticmethod
    def _sent_get_part(sent):
        return sent._.anatomical_part if sent._.anatomical_part else None
    
    def _process_colours_fields(self, sent: Doc, fields: Fields, part: str):
        colour_ents = self._filter_ents(sent, ['COLOUR'])
        for ent in colour_ents:
            fields.upsert(f'{part} colour', 'discrete', ent.lemma_)
            
    def _process_discrete_fields(self, sent: Doc, fields: Fields, part: str, taxon_group: str):        
        df = self.traits.get_discrete_traits(taxon_group)        
        trait_ents = self._filter_ents(sent, ['TRAIT'])
        # Process entities     
        for ent in trait_ents:               
            # Match on either term or character 
            mask = (((df.term == ent.lemma_) | (df.term == ent.text)) | ((df.character == ent.lemma_) | (df.character == ent.text)))
            # If the trait ent is a part, no need to filter on part         
            if part != ent.lemma_:
                # Plant part is an artificial construct, so we treate it as sent having no part             
                if part and part != 'plant':
                    mask &= (df.part == part)
                # If sent has no part, trait must have no part and must be unique             
                else:
                    mask &= ((df.part.isna()) & (df.unique == True))
                
            rows = df[mask]        
            for row in rows.itertuples():   
                fields.upsert(row.trait, 'discrete', row.character)   
                
    def _process_custom_fields(self, sent: Doc, fields: Fields):         
        for ent in sent.ents:
            if trait_value := ent._.get("trait_value"):
                fields.upsert(ent.label_.lower(), 'discrete', trait_value)        
        

    def _process_measurement_fields(self, sent: Doc, fields: Fields, part: str):
        ents = self._filter_ents(sent, ['MEASUREMENT', 'DIMENSION', 'VOLUME'])
        for ent in ents:
            field_type = ent.label_.lower()
            if ent.label_ == 'VOLUME':
                field_name = f'{part} volume'
            else:
                field_name = f'{part} measurement'

            fields.upsert(field_name, field_type, ent)            
            
    def _process_numeric_fields(self, sent: Doc, fields: Fields):        
        # Process numeric values (those not measurements or dimensions)     
        cardinal_ents = self._filter_ents(sent, ['CARDINAL'])       
        # We use the dependency parse to find nummod noun, that's also an entity     
        for cardinal_ent in cardinal_ents:
            root = cardinal_ent.root
            ent = token_get_ent(root.head, ['PART', 'TRAIT'])            
            # displacy.render(sent, style="dep")
            if ent and root.dep_ in ['nummod', 'advcl', 'aux']: 
                field_name = ent._.get("anatomical_part") or ent.lemma_
                fields.upsert(f'{field_name} number', 'numeric', cardinal_ent)
                
    @staticmethod
    def _filter_ents(sent: Span, label: list):
        return [ent for ent in sent.ents if ent.label_ in label]
        
           
# row = df.sample()
# description = row['description'].item()
# text = preprocess(description)  
# doc = nlp(text)


postprocess = Postproccess()  
fields = postprocess(doc, 'angiosperm')

tpl = RAW_DATA_DIR / 'fields.tpl.yml'

print(fields.to_template_dict(tpl))

# for i, row in enumerate(df.itertuples()):
#     text = preprocess(row.description)  
#     try:
#         doc = nlp(text)
#     except:
#         # print(row.description)
#         # print('POST:')
#         # print(text)
#         raise
#     fields = postprocess(doc, 'angiosperm')
#     # print(fields.to_dict())
#     if i > 100:
#         break
                    

OrderedDict([('life form', 'perennial'), ('habitat', None), ('habit', 'tussock, erect leafy/tussock, caespitose, erect leafy'), ('clonality', 'rhizome'), ('perennial organ', 'stem, rhizome'), ('plant min. height [m]', '0.05 m'), ('plant max. height [m]', '0.40 m'), ('indumentum', 'glabrous, pubescent'), ('spinescence', None), ('succulence', None), ('leaf arrangement', None), ('leaf architecture', None), ('leaf position', None), ('leaf shape', None), ('leaf apex', None), ('leaf base', None), ('leaf margin', None), ('leaf min. width [cm]', None), ('leaf max. width [cm]', None), ('leaf min. length [cm]', None), ('leaf max. length [cm]', None), ('inflorescence arrangement', 'spiklet'), ('flower sex', None), ('flower architecture', 'naked'), ('flower merosity', None), ('flower symmetry', None), ('flower shape', None), ('flower colour', 'reddish-brown'), ('petal fusion', None), ('petal colour', None), ('petal min. width [cm]', None), ('petal max. width [cm]', None), ('calyx colour', None), (

In [100]:
x = re.compile(r'(cm|m|mm)+')
list(x.finditer('15-60 (-90) cm'))




[<re.Match object; span=(12, 14), match='cm'>]

In [26]:
from decimal import Decimal, InvalidOperation

In [28]:
def string_to_decimal(string):
    # Remove know characters > < c. ca.     
    string = re.sub(r'>|<|c.|ca.', '', string)    
    try:
        return Decimal(string)
    except InvalidOperation:
        return string  


print(string_to_decimal('c. 23 '))

23


In [19]:
df = pd.read_json(DATA_DIR / 'assets' / 'train_numeric.json')
df.columns = ['sent', 'entities']

In [22]:
df.head()

,sent,entities
0,"A glabrous, loosely to rather densely tufted p...","{'entities': [[55, 68, 'MEASUREMENT']]}"
1,"sheaths all leafless, to 25 cm, yellowish-brow...","{'entities': [[25, 30, 'MEASUREMENT']]}"
2,"Inflorescence an oblong to lanceolate-ovoid, t...","{'entities': [[64, 76, 'MEASUREMENT']]}"
3,"Bristles 0-4, shorter or longer than fruit, so...","{'entities': [[9, 12, 'CARDINAL'], [54, 55, 'C..."
4,Stamens 3;,"{'entities': [[8, 9, 'CARDINAL']]}"


In [5]:
text  = 'A glabrous aquatic or bog plant with the lvs and fls raised above the surface of the water; rhizomes creeping, roots matted. Lvs ternate, lflets obovate or elliptic, 3.5-7 cm, obtuse to subacute, entire, s 7-20 cm, with long sheathing base.  Infl an oblong raceme, scape 12-30 cm, c 10-20 fld; pedicels 5-10 mm, longer than the ovate bracts. Calyx lobes ovate, somewhat recurved. Corolla, campanulate, caducous, deeply 5-lobed, lobes valvate in bud, pink outside, paler or white within, c 15 mm across, much fimbriate within. Stamens 5. Style 1; stigmas 2 lobed.  Capsule dehiscing by 2 lobes. '

doc = nlp(text)



In [27]:
s = """Herbs monoecious, aquatic, sometimes forming dense stands. Stems branched or unbranched, to 3 m. Turions present, becoming brown to red-brown at maturity, clavate to obdeltoid, with abrupt transition from foliage leaves to reduced turion leaves, (6–)11–37(–52) × (3–)4–6(–9) mm, apex ± rounded, lateral turions with several whorls of minute, brown prophylls, entire proximallyand toothed to lobed distally, ovate to elliptic or lance­olate in outline; leaves pectinate, strongly appressed to axis throughout, narrowly flabelliform in outline, 4.5–7.5 × 1.2–1.8(–4) mm; segments 8–12(–18), flattened, linear-lanceolate, longest segment 1.5–6 mm, basal segment usually greater than or equal to 2/3 central axis of leaf, apex ± acute, trichomes usually absent. Leaves in whorls of (3 or)4, heteromorphic; petiole to 6 mm; submersed leaves pectinate, ovate to elliptic in outline, (7–)12–30(–46) × 9–24(–40) mm, segments (9–)12–22(–34), linear-filiform, longest segment (2–)6–19(–29) mm; emersed leaves pectinate, lanceolate to elliptic to ovate in outline, 2–5(–15) ×0.9–2.6 mm, segments (9–)12–20, greater than 0.5 mm. Inflorescences to 25 cm; flowers proximally pistillate, medially bisexual, distally staminate, in whorls of 4; bracteoles cream, ovate, 0.3–0.6(–1) × 0.1–0.6(–1.3) mm, margins deeply dissected into irregular lobes. Staminate flowers: sepals cream, narrowly triangular to deltate, 0.5–0.7(–0.9) × 0.4–0.6(–0.8) mm; petals persistent, cream, sometimes apically suffused with purple, elliptic to obovate, 1.8–2.2(–2.4) ×0.7–1.5 mm; stamens 8, filaments to 2 mm, anthers 0.8–1.7 × 0.3–0.6 mm. Pistillate flowers: sepals greenish to cream, elliptic to triangular, 0.2–0.7 × 0.2–0.7 mm; petals often caducous, sometimes persistent, cream to purple, elliptic to obovate, 0.4–0.7(–0.9) × 0.3–0.5(–0.8) mm; pistils (1.3–)1.8–2.7 mm, stigmas red to± purple, to 0.8 mm. Fruits globose, shallowly 4-lobed. Mericarps olive-green to brown, subglobose to globose, 2–2.7(–3) × (0.9–)1.1–1.3(–1.7) mm, transverselywidely obovate, abaxial surface broadly rounded to moderately flattened, smooth, often with 2 shallow longitudinal ridges, wings and ribs absent. 2n = 28.
Flowering plants of Myriophyllum verticillatum are distinguished from other water-milfoils by reduced, pectinate floral bracts. Many Myriophyllum species can be mistaken for M. verticillatum at early stages of flowering because the proximal floral bracts are transitional in form from submersed leaves and are typically pectinate. The bracteoles also are highly distinctive and are deeply divided into as many as seven narrow lobes. The schizocarps are unique among water-milfoils in the flora area because they have a relatively thick exocarp that typically impedes the splitting and separation of the schizocarp into mericarps at maturity. The most robust and diminutive plants of M. verticillatum can be found in Saskatchewan. No specimens identifiable as M. verticillatum have been seen from New Mexico though it has been reported from there (P. T. Adams 1998). It is likely to occur in New Mexico given that it is found in neighboring states except Oklahoma. The presence and morphology of turions of M. verticillatum can be very helpful in the identification of vegetative material. The clavate to obdeltoid shape and reddish brown color of the turions in this species distinguishes it from the cylindrical, typically dark green turions of M. farwellii and M. sibiricum. 


Four varieties of Myriophyllum verticillatum have been proposed. Fassett based M. verticillatum var. cheneyi solely on the presence of four stamens. M. L. Fernald (1950) considered var. cheneyi conspe­cific with M. hippuroides. Some specimens labeled as var. cheneyi examined during this treatment were confirmed to represent M. hippuroides as noted by S. G. Aiken (1981). Fernald recognized vars. intermedium, pectinatum Wallroth, and pinnatifidum Wallroth based on differences in the length of floral bracts. All of these varieties can be found in the flora area though floral bract length is a very plastic character and there is no distinct separation among the forms. The presence and morphology of turions of M. verticillatum can be very helpful in the identification of vegetative material. The clavate to obdeltoid shape and reddish brown color of the turions in this species differ from the cylindrical, typically dark green turions of both M. farwellii and M. sibiricum."""


In [15]:
from decimal import Decimal

def parse_number(string):

    num_regex = re.compile('[\d\.]+')
    outer_regex = re.compile(r'((?P<lower>[0-9.\/\s]+)\-?\))?\s?(?P<inner>[0-9\-\.]+)\s?(\(\-?(?P<upper>[0-9.\/\s]+))?')
    inner_regex = re.compile(r'(?P<from>[0-9\.]+)?\-?(?P<to>[0-9\.]+)?')
    num = num_regex.findall(string)
    
    if len(num) == 1:
        return Decimal(num[0])
    elif len(num) > 4:
        # If we have more than four number groups, the upper/lower regex will fail
        # If w ehave more than four the ent/text hasn't been parsed correctly
        print("ERROR")
        return
        
    outer_match = outer_regex.search(string)
    match_dict = outer_match.groupdict()
    inner = match_dict.pop('inner')
    inner_match = inner_regex.search(inner)

    match_dict.update(inner_match.groupdict())    
    match_dict = {k: Decimal(v) for k, v in match_dict.items() if v}    
    return match_dict

for ent in doc.ents:
    if ent.label_ not in ['MEASUREMENT', 'CARDINAL']:
        continue
        
    print(ent)
    print(parse_number(ent.text))


# s = 'about 0.2)0-2 (3.000) cm'

# Match optional number in brackets at either end (lower/upper), and the inner

# def 



# print(match_dict)



3.5-7 cm
{'from': Decimal('3.5'), 'to': Decimal('7')}
7-20 cm
{'from': Decimal('7'), 'to': Decimal('20')}
12-30 cm
{'from': Decimal('12'), 'to': Decimal('30')}
10-20
{'from': Decimal('10'), 'to': Decimal('20')}
5-10 mm
{'from': Decimal('5'), 'to': Decimal('10')}
c 15 mm
15
5
5
1
1
2
2
2
2


In [109]:
m = re.search('(?P<from>[0-9\.]+)?\-?(?P<to>[0-9\.]+)?', '2.0-3')
m.groupdict()

{'from': '2.0', 'to': '3'}

In [150]:
s = '(0.1-2)6-9(-12), 2-8(-13) cm'
re.findall('[\d\.]+', s)



['0.1', '2', '6', '9', '12', '2', '8', '13']

In [64]:
import re
from tokenize import Token
from spacy.matcher import Matcher
from spacy.tokens import Span, Doc
from decimal import Decimal, InvalidOperation

from adept.utils.helpers import string_to_decimal
from adept.config import logger, measurement_units, volume_units, unit_registry


# text = 'A glabrous scrambling perennial 60-120 cm. Stem narrowly winged. Leaves pinnate; leaflets 2-3(-5) pairs, 3.5-7 cm, narrowly lanceolate, acute or obtuse and mucronate; tendrils usually branched; stipules 1.5-2 cm, lanceolate, half sagittate. Inflorescence 2-6(-8) fld; peduncles usually longer than leaves. Flowers ca. 12-20 mm, pale purplish-blue; lower calyx-teeth subulate, almost equalling tube. Staminal tube transversely truncate. Style flattened and bearded on its upper side. Fruit a pod 3-5 cm, compressed, glabrous, 3-8 seeded.'

text = 'Perennial herb (10)60-120(140)  cm, nearly (10.2-) 60.3 (-140) glabrous. Roots not tuberous. Radical lvs ±  numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main lflets (smaller ones between); lflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal lflets 3- or occasionally 5-lobed to about 2/3, resembling 3 fused lflets; smaller lflets 1-4 mm; lower stem-lvs resembling radical but shorter and with fewer lflets, uppermost simple or with only small lflets in addition to the terminal one. Stipules broad and toothed  Infl lfy, simple or branched above, glabrous or nearly so of fvery numerous fls in an irregular, cymose panicle usually higher  than broad. Fls sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the fl-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens about twice as long. Carpels 6-12, erect in fl, but soon becoming twisted together spirally, c 2 mm, in fr, glabrous.'

preprocess = Preprocess()

text = preprocess(text)

doc = nlp(text)

class NumericComponent: 
    
    """
    Loop through MEASUREMENT', 'DIMENSION', 'CARDINAL', 'VOLUME' ents, parsing the value.
    
    If numeric range: sets extension numeric_range with keys: lower, from, to, upper
    If not range, sets float numeric_value
    Dimension ents are split into parts in dimension_parts
    
    

    Returns:
        doc: doc object with extensions numeric_range, numeric_value, dimension_parts
    """
    
    # Match digit, and allow a dot, if it's part of at least one digit  
    decimal = '([\d][\d\.]{0,})'
    re_num = re.compile(decimal)
    re_outer = re.compile(r'((?P<lower>[0-9.\/\s]+)\-?\))?\s?(?P<inner>[0-9\-\.]+)\s?(\(\-?(?P<upper>[0-9.\/\s]+))?')
    re_inner = re.compile(r'(?P<from>{0})[\-\s]*(?P<to>{0})?'.format(decimal))
    ent_types = ['MEASUREMENT', 'DIMENSION', 'VOLUME', 'CARDINAL']
    
    units = measurement_units + volume_units    
    units_pattern = '|'.join([unit for unit in units])       
    re_units = re.compile(f'({units_pattern})+') # Include + so mm is preferred to m
    
    def __init__(self, nlp):   
        Span.set_extension("numeric_range", default=[], force=True)
        Span.set_extension("numeric_value", default=[], force=True)
        Span.set_extension("dimension_parts", default=[], force=True)
        Span.set_extension("unit", default=None, force=True)
    
    def __call__(self, doc):        
        for ent in doc.ents:
            if ent.label_ in self.ent_types:
                if ent.label_ == 'DIMENSION':
                    dimensions = self._split_span_on_x(ent, doc)
                    for dimension in dimensions:
                        self._parse_value(dimension)
                    
                    # If the first dimension unit is not set, it is only set at the end 
                    # e.g. 10-20(-40) x (8-)10-30(-35) mm    
                    #  So copy it to the first ent
                    if not dimensions[0]._.unit: dimensions[0]._.unit = dimensions[1]._.unit

                    ent._.set("dimension_parts", dimensions) 
                else:
                    self._parse_value(ent)
                    
        return doc
                                        
    def _parse_value(self, ent: Span):
        text = ent.text
        unit_match = self.re_units.search(text)
        if unit_match: 
            ent._.unit = getattr(unit_registry, unit_match.group())            
            text = self.re_units.sub('', text)
            
        self._parse_numeric(ent)
            
    def _parse_numeric(self, ent):
        
        print(ent)
        
        numeric_parts = self.re_num.findall(ent.text)
        if len(numeric_parts) == 1:
            ent._.set("numeric_value", Decimal(numeric_parts[0])) 
            return
        elif len(numeric_parts) > 4:
            # If we have more than four number groups, the upper/lower regex will fail
            # If w ehave more than four the ent/text hasn't been parsed correctly
            logger.error("Error parsing range %s in %s", ent, ent.sent) 
            return    
        
        # Match outer part, and then inner (lower) inner (upper)
        outer_match = self.re_outer.search(ent.text)
        match_dict = outer_match.groupdict()
        inner = match_dict.pop('inner')
        inner_match = self.re_inner.search(inner)

        match_dict.update(inner_match.groupdict())
        
        print(match_dict)
        
        ent._.set("numeric_range", {k: Decimal(v) for k, v in match_dict.items() if v}) 
        

    @staticmethod
    def _split_span_on_x(ent: Span, doc: Doc):
        x = next((token for token in ent if token.lower_ == 'x'), None)  
        if not x:
            print('NO XXXXXX')
            print(ent)  
            raise Exception('XXXX')       
            
        nbor = x.nbor()
        return [doc[ent.start: x.i], doc[nbor.i: ent.end]]
    
comp = NumericComponent(nlp)

comp(doc)



(10)60-120(140) cm
{'lower': '10', 'upper': '140', 'from': '60', 'to': '120'}
(10.2-) 60.3 (-140)
{'lower': '10.2', 'upper': '140', 'from': '60.3', 'to': None}
30-60 cm
{'lower': None, 'upper': None, 'from': '30', 'to': '60'}
2-5
{'lower': None, 'upper': None, 'from': '2', 'to': '5'}
2-8 cm
{'lower': None, 'upper': None, 'from': '2', 'to': '8'}
3-
0.7
3
1-4 mm
{'lower': None, 'upper': None, 'from': '1', 'to': '4'}
5
5
2-5 mm
{'lower': None, 'upper': None, 'from': '2', 'to': '5'}
6-12
{'lower': None, 'upper': None, 'from': '6', 'to': '12'}
ca. 2 mm


Perennial herb (10)60-120(140) cm, nearly (10.2-) 60.3 (-140) glabrous. Roots not tuberous. Radical leaves +- numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main leaflets (smaller ones between); leaflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal leaflets 3- or occasionally 5-lobed to about 0.7, resembling 3 fused leaflets; smaller leaflets 1-4 mm; lower stem-leaves resembling radical but shorter and with fewer leaflets, uppermost simple or with only small leaflets in addition to the terminal one. Stipules broad and toothed Inflorescence leafy, simple or branched above, glabrous or nearly so of fvery numerous flowers in an irregular, cymose panicle usually higher than broad. Flowers sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the flower-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens a

In [10]:
text

# for sent in doc.sents:
#     print(sent)

'Perennial herb 60-120 cm, nearly glabrous. Roots not tuberous. Radical leaves +- numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main leaflets (smaller ones between); leaflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal leaflets 3- or occasionally 5-lobed to about 0.7, resembling 3 fused lflets; smaller leaflets 1-4 mm; lower stem-leaves resembling radical but shorter and with fewer leaflets, uppermost simple or with only small leaflets in addition to the terminal one. Stipules broad and toothed Inflorescence leafy, simple or branched above, glabrous or nearly so of fvery numerous flowers in an irregular, cymose panicle usually higher than broad. Flowers sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the flower-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens about twice as long. Carpels 6-

In [11]:
preprocess(text)

'Perennial herb 60-120 cm, nearly glabrous. Roots not tuberous. Radical leaves +- numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main leaflets (smaller ones between); leaflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal leaflets 3- or occasionally 5-lobed to about 0.7, resembling 3 fused lflets; smaller leaflets 1-4 mm; lower stem-leaves resembling radical but shorter and with fewer leaflets, uppermost simple or with only small leaflets in addition to the terminal one. Stipules broad and toothed Inflorescence leafy, simple or branched above, glabrous or nearly so of fvery numerous flowers in an irregular, cymose panicle usually higher than broad. Flowers sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the flower-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens about twice as long. Carpels 6-

In [7]:
text = 'Perennial lvs. Lvs herb 60-120 cm, nearly glabrous. Roots not tuberous. Radical leaves +- numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main lflets (smaller ones between); leaflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal lflets 3- or occasionally 5-lobed to about 0.7, resembling 3 lflets fused lflets; smaller leaflets 1-4 mm; lower stem-leaves resembling radical but shorter and with fewer leaflets, uppermost simple or with only small leaflets in addition to the terminal one. Stipules broad and toothed Inflorescence leafy, simple or branched above, glabrous or nearly so of fvery numerous flowers in an irregular, cymose panicle usually higher than broad. Flowers sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the flower-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens about twice as long. Carpels 6-12, erect in flower, but soon becoming twisted together spirally, ca. 2 mm, in fruit, glabrous. diam.'


class PreprocessAbbreviations(object):

    # https://books.google.co.uk/books?id=PoUWAAAAYAAJ&pg=PA128&lpg=PA128&dq=botany+abbreviations+Lvs&source=bl&ots=F2x92m2CMy&sig=ACfU3U3Ptjqi8FpARJirXjnahtCzreg-qA&hl=en&sa=X&ved=2ahUKEwjUiaaq_-vsAhUNesAKHU-KDfQ4ChDoATAFegQIAxAC#v=onepage&q=botany%20abbreviations%20Lvs&f=false

    abbrv = {
        'lf': 'leaf',
        'lvs': 'leaves',
        'lfless': 'leafless',
        'fl': 'flower',
        'fls': 'flowers',
        'flrs': 'flowers',
        'fld': 'flowered',
        'fr': 'fruit',
        'frt': 'fruit',
        'frts': 'fruits',
        'sds': 'seeds',
        'fil': 'filaments',
        'pet': 'petals',
        'infl': 'inflorescence',
        'inf': 'inflorescence',
        'fld': 'flowered',
        'lflets': 'leaflets',
        'lflet': 'leaflet',
        'lfy': 'leafy',
        'lngth': 'length',
        'diam': 'diameter',
    }

    def __init__(self):
        self._re_abbreviations = re.compile(r'(?=\-|\b)(%s)(?:\.|\b)' % '|'.join(
            [f'{k}' for k in self.abbrv.keys()]
        ), re.IGNORECASE)
        self._sentence_end_commas = re.compile(r',(\s+[A-Z])')  

    def __call__(self, text):
        text = self._replace_abbreviations(text)
        text = self._replace_ca(text)
        text = self._remove_double_spaces(text)
        text = self._remove_double_dashes(text)
        text = self._replace_long_dashes(text)
        text = self._replace_sentence_end_commas(text)
        return text

    def _replace_abbreviations(self, text):
        
        def _get_sub_text(match):
            match_text = match.group(1)          
            sub_text = self.abbrv.get(match_text.lower().rstrip('.'), match_text)
            if match_text[0].isupper(): sub_text = sub_text.capitalize()
            return sub_text

        # Pattern: \b(fl\.?|lvs\.?)(?=\s|$)
        # Remove the dot afterwards '{k}\.?' Might need to check for following uppercase char??
        return self._re_abbreviations.sub(_get_sub_text, text)
  
    def _remove_double_spaces(self, text):
        # Quicker than regex - https://stackoverflow.com/questions/1546226/is-there-a-simple-way-to-remove-multiple-spaces-in-a-string
        return " ".join(text.split())

    def _remove_double_dashes(self, text):
        return text.replace('--', '-')
    
    def _replace_ca(self, text):
        # ca 123 => ca. 1234 (otherwise it screws up parsing)
        text = re.sub('\sca\s', ' ca. ', text)
        text = re.sub('\sc\s', ' ca. ', text)
        return text
    
    def _replace_long_dashes(self, text):
        return text.replace('–', '-')  
       
    def _replace_sentence_end_commas(self, text):
        # Replace sentences ending in a comma xxx, Y => xxx. Y
        # E.g. An erect, glabrous, stoloniferous or rhizomatous perennial, 30-60 cm, Stems ... 
        return self._sentence_end_commas.sub(r'.\1', text)   
    
    
# abbrv = PreprocessAbbreviations()

# abbrv(text)

In [58]:
text = 'Perennial herb 60-120 cm, nearly glabrous. Roots not tuberous. Radical leaves +- numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main lflets (smaller ones between); leaflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal lflets 3- or occasionally 5-lobed to about 0.7, resembling 3 lflets fused lflets; smaller leaflets 1-4 mm; lower stem-leaves resembling radical but shorter and with fewer leaflets, uppermost simple or with only small leaflets in addition to the terminal one. Stipules broad and toothed Inflorescence leafy, simple or branched above, glabrous or nearly so of fvery numerous flowers in an irregular, cymose panicle usually higher than broad. Flowers sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the flower-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens about twice as long. Carpels 6-12, erect in flower, but soon becoming twisted together spirally, ca. 2 mm, in fruit, glabrous. diam.'

abbr = re.compile(r'(?=\-|\b)(lf|lvs|lfless|fl|fls|flrs|fld|fr|frt|frts|sds|fil|pet|infl|inf|lflets|lflet|lfy|lngth|diam)(?:\.|\b)', re.IGNORECASE)


abbr.sub('HEY', text)


'Perennial HEY HEY or HEY herb 60-120 cm, nearly glabrous. Roots not tuberous. Radical leaves +- numerous, 30-60 cm including the rather long petiole with 2-5 pairs of main HEY (smaller ones between); leaflets 2-8 cm, ovate, acute, sharply doubly serrate, dark green and glabrous above, usually white-tomentose beneath but frequently green and pubescent or glabrous; terminal HEY 3- or occasionally 5-lobed to about 0.7, resembling 3 HEY fused HEY; smaller leaflets 1-4 mm; lower stem-leaves resembling radical but shorter and with fewer leaflets, uppermost simple or with only small leaflets in addition to the terminal one. Stipules broad and toothed Inflorescence leafy, simple or branched above, glabrous or nearly so of fvery numerous flowers in an irregular, cymose panicle usually higher than broad. Flowers sweet scented. Sepals usually 5, triangular-ovate, reflexed, pubescent (like the flower-stalks). Petals usually 5, 2-5 mm, obovate clawed, cream-white. Stamens about twice as long. Carp

In [30]:
text

'resembling 3 fused lflets ; smaller flrs leaflets 1-4 mm;'

In [92]:
from decimal import Decimal, InvalidOperation
from adept.config import logger, measurement_units, volume_units, unit_registry

def isfloat(num):
    try:
        float(num)        
    except ValueError:
        return False
    else:
        return True

          
        
        
num = NumericComponent(nlp) 

# s = '(0.4-)0.6-1.8'



# isfloat('0')

for i, row in enumerate(df[10:].itertuples()):
    description = row.description
    text = preprocess(description)  
    doc = nlp(text)  
    num(doc)
    if i % 10 == 0:
        print(i)
    


# num._parse_numeric_range('2(-3)')



0


Error parsing range "(70-100) ca. 1 mm": incorrect number of elements
Error parsing range "1.6-3 ' 1.3-2.2 mm": incorrect number of elements


10


KeyboardInterrupt: 

In [48]:
my_list = [1,2,3,2,3,2,3,4]



SyntaxError: expected 'else' after 'if' expression (256805509.py, line 5)

In [46]:
good

[2, 2, 2]

In [47]:
bad

[1, 3, 3, 3, 4]

In [29]:
re.findall(r'[\d]+[\.]{,1}[\d]*', '(2.2-) 3.2 - 4(-7.5)')

['2.2', '3.2', '4', '7.5']